# Semi empirical rudder forces

In [ ]:
# %load imports.py
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

import logging
logger = logging.getLogger('matplotlib.font_manager')
logger.disabled = True

import matplotlib.pyplot as plt
#plt.style.use('./report.mplstyle')  #(comment this one for faster figure generation)

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


import os.path
#import seaman.helpers
#import vct.comparison
#import vct.bokeh_plotting as bokeh_plotting
#from vct.run_seaman import calculate_static_ship, calculate_static_ship_df  
#from vct.comparison import ShipCompareItem 
#from vct.figure_out_test_types import figure_out_test_types
#from seaman.helpers import ShipDict
from sklearn.metrics import r2_score
from collections import OrderedDict

#from mlflow.tracking.client import MlflowClient
#from mlflow.entities import ViewType
#from vessel_manoeuvring_models.to_mlflow import tracking_uri
#mlflow_client = MlflowClient(tracking_uri=tracking_uri)

#import vessel_manoeuvring_models.mlflow_sim as mlflow_sim
#from vessel_manoeuvring_models.mlflow_sim import load_sim_experiment

#import vct.bokeh_plotting as bokeh_plotting
#from vct.run_seaman import calculate_static_ship, calculate_static_ship_df  
from IPython.display import display, Markdown

#from vct.shipdict_compare import load_shipdicts, plot_bars
#from vct.data import load_data
import data
#import vessel_manoeuvring_models.visualization.IMO_standard as imo_standard
import re
import shutil

#from reporting.sspa_report import Appendix,AppendixPart,ModelDescription,Plot, ReportPart
#from pylatex import NoEscape

#import yaml
    
#from vessel_manoeuvring_models.visualization.plotting import track_plot, plot_time_series
#from seaman.evaluation import shipdata_to_latex,pandas_to_latex
#from vct.plot import plotter
#from vessel_manoeuvring_models.visualization.regression import plot_compare
import sympy as sp
from vessel_manoeuvring_models.substitute_dynamic_symbols import lambdify, run

In [ ]:
df_VCT = pd.read_csv('vct.csv')
df_VCT = df_VCT.groupby(by=['model_name']).get_group('V2_5_MDL_modelScale')

In [ ]:
df_rudder = df_VCT.groupby(by='test type').get_group('Rudder angle').sort_values(by='delta')
df_rudder = df_rudder.groupby('V').get_group(df_rudder['V'].unique()[0])

In [ ]:
fig,ax=plt.subplots()
df_rudder.plot(x='delta', y='fy', style='o-', ax=ax)
ax.grid(True)

In [ ]:
V_x = sp.symbols('V_x')
u, v, w, p, q, r = sp.symbols('u v w p q r')
V_xR, V_yR, V_zR = sp.symbols('V_xr V_yr V_zr')
V_xWave, V_yWave, V_zWave = sp.symbols('V_xWave V_yWave V_zWave')
x_R, y_R, z_R = sp.symbols('x_R y_R z_R')
gamma = sp.symbols('gamma')

eq_V_xR_wave = sp.Eq(V_xR, V_x - V_xWave + q*z_R - r*y_R)
eq_V_yR_wave = sp.Eq(V_yR, -v + V_yWave - r*x_R + p*z_R)
eq_V_zR_wave = sp.Eq(V_zR, -w + V_zWave - q*y_R - q*x_R)

display(eq_V_xR_wave)
display(eq_V_yR_wave)
display(eq_V_zR_wave)


In [ ]:
eq_V_xR = eq_V_xR_wave.subs(V_xWave,0)
eq_V_yR = eq_V_yR_wave.subs(V_yWave,0)
eq_V_zR = eq_V_zR_wave.subs(V_zWave,0)

display(eq_V_xR)
display(eq_V_yR)
display(eq_V_zR)

In [ ]:
eq_gamma = sp.Eq(gamma, sp.atan(V_yR/V_xR))
eq_gamma

In [ ]:
V_R = sp.symbols('V_R')
eq_V_R = sp.Eq(V_R, sp.sqrt(V_xR**2+V_yR**2+V_zR**2))
eq_V_R

In [ ]:
A,A_g = sp.symbols('Lambda Lambda_g')
delta,delta_lim = sp.symbols('delta delta_lim')

In [ ]:
L,D,rho,C_L,C_D,A_R,b = sp.symbols('L,D,rho,C_L,C_D,A_R,b')

In [ ]:
eq_A_g = sp.Eq(A_g,b**2/A_R)
eq_A_g

In [ ]:
eq_A = sp.Eq(A, A_g*(2-sp.Abs(delta/delta_lim)))
eq_A

In [ ]:
eq_L = sp.Eq(L, 1/2*rho*C_L*A_R*V_R**2)
eq_D = sp.Eq(D, 1/2*rho*C_D*A_R*V_R**2)
display(eq_L)
display(eq_D)

In [ ]:
eq_C_L = sp.Eq(C_L, 2*sp.pi*A*(A+1)/(A+2)**2*sp.sin(delta+gamma))
eq_C_L

In [ ]:
C_D0 = sp.symbols('C_D0')
eq_C_D = sp.Eq(C_D, 1.1*C_L**2/(sp.pi*A)+C_D0)
eq_C_D

In [ ]:
C_F,R_e = sp.symbols('C_F,R_e')
eq_C_D0 = sp.Eq(C_D0, 2.5*C_F)
eq_C_D0

In [ ]:
eq_CF = sp.Eq(C_F, 0.075/((sp.log(R_e)-2)**2))
eq_CF

In [ ]:
kinematic_viscosity,c = sp.symbols('nu c')
eq_Re = sp.Eq(R_e, V_R*c/kinematic_viscosity)
eq_Re

In [ ]:
lambdify_equations = [
eq_V_xR,
eq_V_yR,
eq_V_zR,
eq_gamma,
eq_V_R,
eq_A_g,
eq_A,
eq_C_L,
eq_Re,
eq_CF,
eq_C_D0,
eq_C_D,
eq_L,
eq_D,
]

lambdas = {eq.lhs:lambdify(eq.rhs) for eq in lambdify_equations}

In [ ]:
lambdas

In [ ]:
shipdict = ShipDict.load('V2_5_MDL_modelScale_R11.ship')
list(shipdict.keys())

In [ ]:
rudder_particulars = shipdict['rudder_particulars'][0]
rudder_particulars

In [ ]:
df = df_rudder[['u','v','r','p','rho','delta','thrust']].copy()
df['q'] = 0
df['w'] = 0
wf = shipdict['fix_prop_data'][0]['wffix']
df['V_x'] = (1-wf)*df['u']  # propeller induced speed...
df['x_R'] = rudder_particulars['xxrud']
df['y_R'] = rudder_particulars['yyrud']
df['z_R'] = rudder_particulars['zzrud']
df['A_R'] = rudder_particulars['ar']
df['b'] = 8.79
df['delta_lim'] = np.deg2rad(45)
df['c'] = df['A_R']/df['b']
df['nu'] = 1.18849E-06

for symbol, function in lambdas.items():
    
    df[str(symbol)] = run(function=function, inputs=df)

In [ ]:
fig,ax=plt.subplots()
df_rudder['delta_deg'] = np.rad2deg(df_rudder['delta'])
df_rudder.plot(x='delta_deg', y='fy', style='o-', label='ShipFlow', ax=ax)
ax.grid(True)

n_rud = len(shipdict['rudder_particulars'])
df['fy'] = n_rud*df['L']
df['delta_deg'] = np.rad2deg(df['delta'])
df.plot(x='delta_deg', y='fy', label='Semi empirical', ax=ax);
ax.grid(True)
ax.set_ylabel('fy')
ax.set_xlabel('Rudder angle [deg]');
ax.set_title('WPCC Rudder angle variation');